In [0]:
import numpy as np
import csv
import pandas as pd
import nltk
import random
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download("popular")
from scipy import sparse

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#   print('title: %s' % (file1['title']))

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
  
downloaded = drive.CreateFile({'id':'1A6SqtCXt81Lqhoph0iWNOAd0HoI_GYAW'}) 
downloaded.GetContentFile('reddit_train.csv')    

reddit_train = pd.read_csv('reddit_train.csv')

# print(len(reddit_train))
# print(reddit_train.iloc[0][1])


# tokenize the comments
def loadToken(reddit_train):
  reddit_tokens = []
  for i in range(0,len(reddit_train)):
    entry = word_tokenize(reddit_train.iloc[i][1])
    print(entry)
    reddit_tokens.append(entry)

  #print(len(reddit_tokens))  
  #sum_preclean_tokens = sum(len(entry) for entry in reddit_tokens)
  #print(sum_preclean_tokens)
  
  return reddit_tokens
#all words in comments including puntuation marks

In [0]:
# clean data for processing
def clean(entry):

  # delete non-alphabetical tokens and lowercases
  tokensA = [token.lower() for token in entry if token.isalpha()]
  # delete stopwords
  stopword = set(stopwords.words('english'))
  #print(stopword)
  tokensB = [token for token in tokensA if token not in stopword]

  # lemmatization
  output = [WordNetLemmatizer().lemmatize(token) for token in tokensB]
    
  return output


['tfidf']

In [0]:
pre_tokens = loadToken(reddit_train)

cleaned_tokens = [clean(entry) for entry in pre_tokens]

calSum = sum([sum(len(x) for x in cleaned_tokens)])
print(calSum)  # total number of tokens: 1452983

print(len(cleaned_tokens))

print(cleaned_tokens[0])
#print(cleaned_tokens[69999])

1452983
70000
['honestly', 'buffalo', 'correct', 'answer', 'remember', 'people', 'somewhat', 'joking', 'buffalo', 'mantra', 'starting', 'goalie', 'win', 'game', 'get', 'traded', 'think', 'edmonton', 'front', 'office', 'travesty', 'better', 'part', 'year', 'buffalo', 'systematic', 'destruction', 'term', 'much', 'responsible', 'change', 'draft', 'lottery']


In [0]:
#partition dataset to train and test in a ratio(testPercent)
def partitionData(cleanedData, testPercent):
    howManyNumbers = int(round(testPercent*len(cleanedData)))
    #shuffled = cleanedData[:]
    #random.shuffle(shuffled)
    test_dataset=cleanedData[howManyNumbers:]
    train_dataset=cleanedData[:howManyNumbers]
    return train_dataset,test_dataset

In [0]:
# create a word bank (removed repeated words) (only for training data)
def wordBankCreator(cleaned_tokens):

  ResultList = list()
  for entry in cleaned_tokens:
    for token in entry:
      if token not in ResultList:
        ResultList.append(token)
        
  return ResultList     

In [0]:
# extract a list if subreddit categories and match each with number index
def encodeType(reddit_train):
  typeList = list()
  
  # extract subreddits
  for i in range(0,len(reddit_train)):  
    subreddit = reddit_train.iloc[i][2]
    #subreddit = reddit_train[i][2] 
    if subreddit not in typeList:
      typeList.append(subreddit)    
    # if(len(typeList)>=20):
    #   break;
    
  # index subreddits  
  encoded = list()
  index = 0;
  for element in typeList:
    my_tuple = (index,element)
    index = index+1
    encoded.append(my_tuple)
    
  return encoded

In [0]:
# return vector that indicates subreddit category
def typeSubreddit(reddit_train,subreddit_indexed):
  
  typeVector = list()
  
  for i in range(0,len(reddit_train)):  
    subreddit = reddit_train.iloc[i][2]
    
    for k in range(0,len(subreddit_indexed)):
      if subreddit_indexed[k][1] == subreddit:
        typeVector.append(subreddit_indexed[k][0])
   
  return typeVector

In [0]:
train_dataset,test_dataset=partitionData(cleaned_tokens,0.9)
wordBank = wordBankCreator(train_dataset)
print(len(wordBank))   # total number in word bank: 55145 

subreddit_indexed = encodeType(reddit_train)
print(subreddit_indexed)  # correctly gives 20

subreddit_type = typeSubreddit(reddit_train,subreddit_indexed)
train_type,test_type=partitionData(subreddit_type,0.9)
print(len(train_type))   # correctly gives 70000   

52410
[(0, 'hockey'), (1, 'nba'), (2, 'leagueoflegends'), (3, 'soccer'), (4, 'funny'), (5, 'movies'), (6, 'anime'), (7, 'Overwatch'), (8, 'trees'), (9, 'GlobalOffensive'), (10, 'nfl'), (11, 'AskReddit'), (12, 'gameofthrones'), (13, 'conspiracy'), (14, 'worldnews'), (15, 'wow'), (16, 'europe'), (17, 'canada'), (18, 'Music'), (19, 'baseball')]
63000


In [0]:
# return X matrix: for each comment, represent occurance of each word in wordBank with 1 or 0
# outcome expected - sparse matrix: 70000 * 55145
def BernoulliMatrix(cleaned_tokens,wordBank):

  matrixResult = sparse.lil_matrix((len(cleaned_tokens),len(wordBank)),dtype=np.int8)
  
  print(matrixResult.shape)
  print(len(cleaned_tokens))
  print(len(wordBank))      
  
  for i in range(0,len(cleaned_tokens)):
    
    comment_cleaned = cleaned_tokens[i]

    for j in range(0,len(comment_cleaned)):
      
      if comment_cleaned[j] in wordBank:
        occurIndex = wordBank.index(comment_cleaned[j])
        matrixResult[i,occurIndex] = 1
      
    
  return matrixResult
    
    
    
train_matrixResult = BernoulliMatrix(train_dataset,wordBank)
test_matrixResult = BernoulliMatrix(test_dataset,wordBank)
#print(matrixResult.shape)


(63000, 52410)
63000
52410
(7000, 52410)
7000
52410


In [0]:
 #Bernoulli Naive Bayes

import numpy as np

class BernNB() :
  
  def __init__(self):
    c = 20
    

  def fit(self, X, Y, c):
    m = X.shape[1]
    pY1 = np.zeros((c,), dtype = np.float_)
    xJy = np.zeros((c, m), dtype = np.float_)
    n = X.shape[0]
    
    for i in range(0, n):
      numClass = Y[i]
      pY1[numClass] = pY1[numClass] + 1
    
    print("000000000000")

    for j in range(0, n):
      for k in range(0, m):
        if X[j, k] == 1:
          xJy[Y[j], k] = xJy[Y[j], k] + 1
    
    print("000000000000")

    for i in range(0, c):
      yAdd2 = pY1[i] + 2
      for j in range(0, m):
        xJy[i, j] = (xJy[i, j] + 1)/yAdd2
    
    pY1 * (1/n)
    self.pY1 = pY1
    self.xJy = xJy


  def predict(self, X):
    n = X.shape[0]
    m = X.shape[1]
    c = self.xJy.shape[0]
    totRes = np.zeros((n,))

    for i in range(0, n):
      maxY = 0
      maxProb = 0
      for j in range(0, c):
        prob = self.pY1[j]
        for k in range(0, m):
          if X[i, k] == 1:
            prob = prob * self.xJy[j, k]
          else:
            prob = prob * (1 - self.xJy[j, k])
        if prob > maxProb:
          maxY = j
          maxProb = prob
      totRes[i] = maxY
    
    return totRes
 

In [0]:
BNB = BernNB()
BNB.fit(train_matrixResult.toarray(),train_type,20)
result = BNB.predict(test_matrixResult.toarray())

if result.shape[0] != len(test_type):
  print("出问题了")
else:
  accu = 0
  for i in range(0, result.shape[0]):
    if result[i] == test_type[i]:
      accu = accu + 1
  
  print(accu/result.shape[0])



000000000000
000000000000
0.5034285714285714


5w of wordBank is too big;

the predicted output should be stored back in test dataset (number->category)